In [1]:
# Append web folder to system path
import sys
sys.path.append('../web')

In [2]:
from db_config import Base
from Database import Station, Availability, Weather, WeatherPredictive
from sqlalchemy import create_engine, func, Column, String, Integer, Double, Boolean, MetaData
from sqlalchemy.orm import sessionmaker
import json
import sys
import requests
from datetime import datetime, timedelta

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

In [4]:
# Get the db_info
with open('../dbinfo.json') as f:
    db_info = json.load(f)


USER = db_info['dbConnection']['USER']
PASSWORD = db_info['dbConnection']['PASSWORD']
URI = db_info['dbConnection']['URI']
PORT = db_info['dbConnection']['PORT']
DB = db_info['dbConnection']['DB']
WEATHER_API_KEY = db_info['weatherKey']
FORECAST_URI = 'https://api.openweathermap.org/data/2.5/forecast'


# Create a new session
engine = create_engine(
    'mysql+pymysql://{}:{}@localhost:{}/{}'.format(USER, PASSWORD, PORT, DB), echo=True)
Base.metadata.create_all(bind=engine)
Session = sessionmaker(bind=engine)
session = Session()
print("connected")

2024-04-18 22:38:58,468 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-04-18 22:38:58,471 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-18 22:38:58,545 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-04-18 22:38:58,560 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-18 22:38:58,602 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-04-18 22:38:58,607 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-18 22:38:58,679 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-18 22:38:58,681 INFO sqlalchemy.engine.Engine DESCRIBE `db_on_your_bike`.`stations`
2024-04-18 22:38:58,682 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-18 22:38:58,722 INFO sqlalchemy.engine.Engine DESCRIBE `db_on_your_bike`.`availability`
2024-04-18 22:38:58,723 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-18 22:38:58,757 INFO sqlalchemy.engine.Engine DESCRIBE `db_on_your_bike`.`weather`
2024-04-18 22:38:58,761 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-18 22:38:5

In [5]:
weather_historical = session.query(Weather).filter(
    Weather.time_updated > midnight).all()

weather_historical_df = pd.DataFrame(
    [row.__dict__ for row in weather_historical])

weather_historical_df = weather_historical_df[[
    'time_updated', 'temperature', 'wind_speed', 'humidity', 'type']]


NameError: name 'midnight' is not defined

In [6]:
station_id = 1
midnight = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)

station_information = session.query(Availability).filter(
Availability.time_updated > midnight, Availability.station_id == station_id).all()

station_data = pd.DataFrame([row.__dict__ for row in station_information])
station_data

2024-04-18 22:45:40,367 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-18 22:45:40,395 INFO sqlalchemy.engine.Engine SELECT availability.station_id AS availability_station_id, availability.time_updated AS availability_time_updated, availability.bike_stands AS availability_bike_stands, availability.available_bikes AS availability_available_bikes, availability.available_bike_stands AS availability_available_bike_stands, availability.status AS availability_status 
FROM availability 
WHERE availability.time_updated > %(time_updated_1)s AND availability.station_id = %(station_id_1)s
2024-04-18 22:45:40,397 INFO sqlalchemy.engine.Engine [generated in 0.00176s] {'time_updated_1': datetime.datetime(2024, 4, 18, 0, 0), 'station_id_1': 1}


,_sa_instance_state,available_bikes,time_updated,status,available_bike_stands,station_id,bike_stands
0,<sqlalchemy.orm.state.InstanceState object at ...,0,2024-04-18 00:00:07,OPEN,31,1,31
1,<sqlalchemy.orm.state.InstanceState object at ...,0,2024-04-18 00:05:04,OPEN,31,1,31
2,<sqlalchemy.orm.state.InstanceState object at ...,0,2024-04-18 00:10:03,OPEN,31,1,31
3,<sqlalchemy.orm.state.InstanceState object at ...,0,2024-04-18 00:15:03,OPEN,31,1,31
4,<sqlalchemy.orm.state.InstanceState object at ...,0,2024-04-18 00:20:03,OPEN,31,1,31
...,...,...,...,...,...,...,...
258,<sqlalchemy.orm.state.InstanceState object at ...,0,2024-04-18 21:25:04,OPEN,31,1,31
259,<sqlalchemy.orm.state.InstanceState object at ...,0,2024-04-18 21:30:05,OPEN,31,1,31
260,<sqlalchemy.orm.state.InstanceState object at ...,0,2024-04-18 21:35:04,OPEN,31,1,31
261,<sqlalchemy.orm.state.InstanceState object at ...,0,2024-04-18 21:40:04,OPEN,31,1,31


In [7]:
station_data = station_data.groupby(station_data['time_updated'].dt.floor('H')).agg({
'available_bikes': 'mean',  # Example: Calculate mean temperature for each hour
'bike_stands': 'mean'      # Example: Calculate mean humidity for each hour
}).reset_index()
station_data['hour'] = station_data['time_updated'].dt.hour
station_data = station_data[['hour', 'available_bikes', 'bike_stands']]
station_data


/var/folders/z2/g7zs32wn0h7gwvz3dx_mgqpm0000gn/T/ipykernel_58627/1439544802.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  station_data = station_data.groupby(station_data['time_updated'].dt.floor('H')).agg({


,hour,available_bikes,bike_stands
0,0,0.000000,31.0
1,1,0.000000,31.0
2,2,0.000000,31.0
3,3,0.000000,31.0
4,4,0.416667,31.0
5,5,4.583333,31.0
6,6,5.833333,31.0
7,7,7.333333,31.0
8,8,12.666667,31.0
9,9,16.750000,31.0


In [8]:
weather_predictive = session.query(WeatherPredictive).all()
weather_predictive_df = pd.DataFrame(
        [row.__dict__ for row in weather_predictive])
weather_predictive_df['type'] = weather_predictive_df['weather_type']
weather_predictive_df = weather_predictive_df[[
        'time_updated', 'temperature', 'wind_speed', 'humidity', 'type']]

weather_predictive_df

2024-04-18 22:45:40,577 INFO sqlalchemy.engine.Engine SELECT weather_predictive.time_updated AS weather_predictive_time_updated, weather_predictive.type AS weather_predictive_type, weather_predictive.description AS weather_predictive_description, weather_predictive.temperature AS weather_predictive_temperature, weather_predictive.feels_like AS weather_predictive_feels_like, weather_predictive.min_temp AS weather_predictive_min_temp, weather_predictive.max_temp AS weather_predictive_max_temp, weather_predictive.rain_3h AS weather_predictive_rain_3h, weather_predictive.humidity AS weather_predictive_humidity, weather_predictive.wind_speed AS weather_predictive_wind_speed, weather_predictive.clouds AS weather_predictive_clouds 
FROM weather_predictive
2024-04-18 22:45:40,585 INFO sqlalchemy.engine.Engine [generated in 0.00740s] {}


,time_updated,temperature,wind_speed,humidity,type
0,2024-04-19 00:00:00,10.50,5.62,84,Clouds
1,2024-04-19 03:00:00,10.20,5.41,89,Rain
2,2024-04-19 06:00:00,9.24,4.10,94,Rain
3,2024-04-19 09:00:00,10.95,5.29,84,Clouds
4,2024-04-19 12:00:00,10.74,4.90,83,Rain
5,2024-04-19 15:00:00,12.87,5.37,64,Rain
6,2024-04-19 18:00:00,11.53,3.58,68,Clouds
7,2024-04-19 21:00:00,6.49,2.21,89,Clouds
8,2024-04-20 00:00:00,3.33,1.85,89,Clouds
9,2024-04-20 03:00:00,2.36,1.40,92,Clouds


In [12]:
# Generate a list of hours for today
current_date = datetime.now()
current_hour = current_date.hour
hours_today = [current_date.replace(
    hour=h, minute=30, second=0, microsecond=0) for h in range(current_hour, 24)]
hourly_df = pd.DataFrame(hours_today, columns=['time_updated'])

weather_df = pd.merge_asof(hourly_df, weather_predictive_df, on='time_updated', direction='nearest')
weather_df

,time_updated,temperature,wind_speed,humidity,type
0,2024-04-18 22:30:00,10.5,5.62,84,Clouds
1,2024-04-18 23:30:00,10.5,5.62,84,Clouds


In [13]:
days = ['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday',
            'Wednesday']
# One hot encode day of the week
weather_df['weekday'] = weather_df['time_updated'].dt.day_name(
)
weather_df['day'] = weather_df['time_updated'].dt.day
weather_df['hour'] = weather_df['time_updated'].dt.hour
for day in days:
    weather_df[day] = weather_df['weekday'] == day

weather_df['rain'] = weather_df['type'] == 'Rain'

weather_df.drop('time_updated', axis=1, inplace=True)
weather_df.drop('weekday', axis=1, inplace=True)
weather_df.drop('type', axis=1, inplace=True)

# Get the columns in the right order
weather_df = weather_df[['temperature', 'wind_speed', 'humidity', 'hour', 'day', 'rain',
       'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday',
       'Wednesday']]

In [14]:
weather_predictive_df

,time_updated,temperature,wind_speed,humidity,type
0,2024-04-19 00:00:00,10.50,5.62,84,Clouds
1,2024-04-19 03:00:00,10.20,5.41,89,Rain
2,2024-04-19 06:00:00,9.24,4.10,94,Rain
3,2024-04-19 09:00:00,10.95,5.29,84,Clouds
4,2024-04-19 12:00:00,10.74,4.90,83,Rain
5,2024-04-19 15:00:00,12.87,5.37,64,Rain
6,2024-04-19 18:00:00,11.53,3.58,68,Clouds
7,2024-04-19 21:00:00,6.49,2.21,89,Clouds
8,2024-04-20 00:00:00,3.33,1.85,89,Clouds
9,2024-04-20 03:00:00,2.36,1.40,92,Clouds


In [17]:
with open(f'../ML_models/station_{station_id}.pkl', 'rb') as file:
    # Load the model from the file
    poly_reg_model = pickle.load(file)

poly = PolynomialFeatures(degree=3, include_bias=False)
poly_features = poly.fit_transform(weather_df)


In [18]:

weather_predictive_df['predicted_available'] = poly_reg_model.predict(poly_features)
weather_predictive_df

ValueError: Length of values (2) does not match length of index (40)

In [ ]:
weather_predictive_df.loc[:,'predicted_available'] = np.clip(
    weather_predictive_df['predicted_available'], 0, station_data['bike_stands'][0])
weather_predictive_df

,temperature,wind_speed,humidity,hour,rain,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,predicted_available
0,9.52,4.31,89,11,False,False,False,False,False,True,False,False,16.058165
1,9.52,4.31,89,12,False,False,False,False,False,True,False,False,15.701905
2,9.52,4.31,89,13,False,False,False,False,False,True,False,False,15.616233
3,9.93,4.83,86,14,True,False,False,False,False,True,False,False,18.595520
4,9.93,4.83,86,15,True,False,False,False,False,True,False,False,19.311704
5,9.93,4.83,86,16,True,False,False,False,False,True,False,False,20.229542
6,9.35,3.20,97,17,True,False,False,False,False,True,False,False,27.180671
7,9.35,3.20,97,18,True,False,False,False,False,True,False,False,28.457760
8,9.35,3.20,97,19,True,False,False,False,False,True,False,False,29.994848
9,9.47,4.46,98,20,True,False,False,False,False,True,False,False,30.595550


In [ ]:
combined_df = pd.concat(
    [station_data[['hour', 'available_bikes']], weather_predictive_df[['hour', 'predicted_available']]])
combined_df.replace(np.nan, None, inplace=True)
combined_df

,hour,available_bikes,predicted_available
0,0,2.0,None
1,1,2.0,None
2,2,2.0,None
3,3,2.0,None
4,4,3.0,None
5,5,6.833333,None
6,6,11.0,None
7,7,10.0,None
8,8,15.916667,None
9,9,17.833333,None


In [ ]:
data = {'data': []
        }
for hour, avail_bikes, pred_avail in zip(combined_df['hour'].values.tolist(), combined_df['available_bikes'].values.tolist(), combined_df['predicted_available'].values.tolist()):
    data['data'].append([str(hour), avail_bikes, pred_avail])
print(data, file=sys.stdout)

{'data': [['0', 2.0, None], ['1', 2.0, None], ['2', 2.0, None], ['3', 2.0, None], ['4', 3.0, None], ['5', 6.833333333333333, None], ['6', 11.0, None], ['7', 10.0, None], ['8', 15.916666666666666, None], ['9', 17.833333333333332, None], ['10', 20.142857142857142, None], ['11', None, 16.058165298252447], ['12', None, 15.701904648392542], ['13', None, 15.616233322247012], ['14', None, 18.595520436241372], ['15', None, 19.311704117282375], ['16', None, 20.229542027636512], ['17', None, 27.1806713514394], ['18', None, 28.457760099730535], ['19', None, 29.99484814558093], ['20', None, 30.5955502814046], ['21', None, 31.0], ['22', None, 31.0], ['23', None, 31.0]]}


In [ ]:
combined_df = pd.concat([station_data[['hour', 'available_bikes']], weather_predictive_df[['hour', 'predicted_available']]])
combined_df.replace(np.nan, None, inplace=True)
combined_df


,hour,available_bikes,predicted_available
0,0,2.0,None
1,1,2.0,None
2,2,2.0,None
3,3,2.0,None
4,4,3.0,None
5,5,6.833333,None
6,6,11.0,None
7,7,10.0,None
8,8,15.916667,None
9,9,17.833333,None


In [ ]:
data = {'data': []
        }
for hour, avail_bikes, pred_avail in zip(combined_df['hour'].values.tolist(), combined_df['available_bikes'].values.tolist(), combined_df['predicted_available'].values.tolist()):
    data['data'].append([str(hour), avail_bikes, pred_avail])
print(data, file=sys.stdout)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
{'data': [['0', 2.0, None], ['1', 2.0, None], ['2', 2.0, None], ['3', 2.0, None], ['4', 3.0, None], ['5', 6.833333333333333, None], ['6', 11.0, None], ['7', 10.0, None], ['8', 15.916666666666666, None], ['9', 17.833333333333332, None], ['10', 20.142857142857142, None], ['11', None, 16.058165298252447], ['12', None, 15.701904648392542], ['13', None, 15.616233322247012], ['14', None, 18.595520436241372], ['15', None, 19.311704117282375], ['16', None, 20.229542027636512], ['17', None, 27.1806713514394], ['18', None, 28.457760099730535], ['19', None, 29.99484814558093], ['20', None, 30.5955502814046], ['21', None, 31.0], ['22', None, 31.0], ['23', None, 31.0]]}


,temperature,wind_speed,humidity,hour,rain,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,predicted_available
0,8.18,5.66,92,0,True,False,False,False,False,False,False,True,0.042450
1,8.21,6.69,91,1,True,False,False,False,False,False,False,True,1.008216
2,8.18,6.69,93,2,True,False,False,False,False,False,False,True,0.359445
3,8.36,6.17,94,3,False,False,False,False,False,False,False,True,8.298418
4,8.46,6.17,93,4,True,False,False,False,False,False,False,True,4.162119
5,8.46,7.72,94,5,True,False,False,False,False,False,False,True,2.920615
6,8.05,6.69,95,6,True,False,False,False,False,False,False,True,3.863218
7,7.50,7.20,93,7,False,False,False,False,False,False,False,True,11.260365
8,7.37,7.20,93,8,False,False,False,False,False,False,False,True,11.951043
9,7.53,7.72,94,9,False,False,False,False,False,False,False,True,12.420441
